In [8]:
# The goal of this assignment is to the code to compute the velocity dispersion
# within the Jacobi Radius of M33 over time and plot this evolution.

In [9]:
# numpy provides powerful multi-dimensional arrays to hold and manipulate data
import numpy as np
# astropy provides unit system for astronomical calculations
import astropy.units as u
# import previous HW functions
from ReadFile import Read

In [10]:
# Rj = r*(Msat/2/Mmw)**(1/3)
# This function is a modified function from Lab 4 from class
def jacobi_radius(m_sat,r,m_host):
    """ Function that determines the jacobi radius for a satellite 
    on a circular orbit about a host, where the host
    is assumed to be an isothermal sphere halo
    
    Inputs:
        m_sat : astropy quantity
            Mass of the satellite galaxy in Msun 
        r : astropy quantity 
            Distance of the satellite from the host in kpc
        m_host: astropy quantity 
            Mass of the host galaxy in Msun within r in Msun
        
    Outputs:
        jacobi_radius: astropy quantity
            The radius at which a satellite can be disturbed by 
            tidal forces of the host galaxy
    """
    Rj = r*(m_sat/(2*m_host))**(1/3)
    return Rj

In [11]:
# Function to calculate velocity dispersion for particles within a specific radius
def velocity_dispersion_within_radius(vx, vy, vz, positions, r_jacobi):
    """ Calculate the velocity dispersion for particles within a specific radius    
    Inputs:
        vx, vy, vz: array
            Velocity components of the particles (km/s)
        positions: array
            Position vectors of the particles (kpc)
        r_jacobi: astropy quantity
            Jacobi radius
    
    Outputs:
        sigma: float
            Velocity dispersion for particles within the Jacobi radius
    """
    # Calculate the distance of each particle from host galaxy
    distances = np.linalg.norm(positions, axis=1)
    
    # Select particles within the Jacobi radius
    mask = distances <= r_jacobi
    selected_vx = vx[mask]
    selected_vy = vy[mask]
    selected_vz = vz[mask]
    
    # Number of selected particles
    N = len(selected_vx)
    
    # Calculate the mean velocities of the given particles
    vmean_x = np.mean(selected_vx)
    vmean_y = np.mean(selected_vy)
    vmean_z = np.mean(selected_vz)
    
    # Calculate the squared differences from the mean for each component
    dispersion_x = np.mean((selected_vx - vmean_x)**2)
    dispersion_y = np.mean((selected_vy - vmean_y)**2)
    dispersion_z = np.mean((selected_vz - vmean_z)**2)
    
    # Calculate the total velocity dispersion
    sigma = np.sqrt(dispersion_x + dispersion_y + dispersion_z)
    
    return sigma

In [12]:
# Function to plot the velocity dispersion
def plot_velocity_dispersion(start_num, end_num, m_sat, r, m_host, prefix="M33_", suffix=".txt"):
    """
    Loops through multiple files with a specified prefix, calculates the velocity dispersion 
    within the Jacobi radius, and plots the results.
    
    Inputs:
        start_num, end_num: int
            Range of file numbers
        m_sat: astropy quantity
            Mass of the satellite galaxy in Msun
        r: astropy quantity
            Distance of the satellite from the host in kpc
        m_host: astropy quantity
            Mass of the host galaxy in Msun within r in Msun
        prefix: str
            The prefix for the filename 
        suffix: str
            The type of file
    
    Outputs:
        A plot
    """
    times = []
    dispersions = []

    # Loop over the range of file numbers
    for i in range(start_num, end_num + 1):
        filename = f"{prefix}{i:03d}{suffix}"

        time, total, data = Read(filename)

        # Calculate Jacobi radius for M33
        r_jacobi = jacobi_radius(m_sat, r, m_host)

        # Extract velocity components and positions
        vx = data['vx']
        vy = data['vy']
        vz = data['vz']
        # From ChatGPT
        positions = np.column_stack((data['x'], data['y'], data['z']))

        # Calculate the velocity dispersion
        sigma = velocity_dispersion_within_radius(vx, vy, vz, positions, r_jacobi)

        # Store the time and velocity dispersion
        times.append(time.value)  # Convert time to a simple float (in Myr)
        dispersions.append(sigma)

    # Plotting the results
    plt.figure(figsize=(10, 6))
    plt.plot(times, dispersions, marker='o', linestyle='-', color='b')
    plt.xlabel('Time (Myr)')
    plt.ylabel('Velocity Dispersion (km/s)')
    plt.title(f'Velocity Dispersion Within Jacobi Radius vs Time')
    plt.grid(True)
    plt.show()

start_num = 0    
end_num = 99   
m_sat = 19.6e10 * u.Msun  
r = 100 * u.kpc  
m_host = 206e10 * u.Msun  

plot_velocity_dispersion(start_num, end_num, m_sat, r, m_host)


FileNotFoundError: [Errno 2] No such file or directory: 'M33_000.txt'